In [ ]:
import math

In [ ]:
x = [1, -1]
b = [0., 0., 0.]
c = [0., 0.]
w = [[1., 1., 1.], [-1., -1., -1.]]
v = [[1., 1.], [-1., -1.], [-1., -1.]]
t = [1, 0]
k1 = [0.] * 3
k2 = [0.] * 2

In [ ]:
# softmax activation
def softmax(x):
    e_x = [math.exp(i) for i in x]
    sum_e_x = sum(e_x)
    return [i / sum_e_x for i in e_x]

def forward_propagation(x, w, b, v, c):
  
  for j in range(len(k1)):
    for i in range(len(x)):
      k1[j] += w[i][j] * x[i]
    k1[j] += b[j]

  # sigmoid activation
  h = [1 / (1 + math.exp(-1*k1[i])) for i in range(len(k1))]

  # k2 = vh + c
  
  for j in range(len(k2)):
    for i in range(len(h)):
      k2[j] += v[i][j] * h[i]
    k2[j] += c[j]

  y = softmax(k2)

  return y, h

y, h = forward_propagation(x, w, b, v, c)

In [ ]:
def back_propagation(x, h, y, t):
    dL_dk2 = [y[i] - t[i] for i in range(len(y))]

    dL_dv = [[dL_dk2[j] * h[i] for j in range(len(dL_dk2))] for i in range(len(h))]

    dL_dc = dL_dk2

    dL_dh = [sum(dL_dk2[j] * v[i][j] for j in range(len(dL_dk2))) for i in range(len(h))]
    dh_dk1 = [h[i] * (1 - h[i]) for i in range(len(h))]  # derivative of sigmoid function

    dL_dk1 = [dL_dh[i] * dh_dk1[i] for i in range(len(h))]

    dL_dw = [[dL_dk1[j] * x[i] for j in range(len(dL_dk1))] for i in range(len(x))]

    dL_db = dL_dk1

    return dL_dv, dL_dc, dL_dw, dL_db

In [ ]:
dL_dv, dL_dc, dL_dw, dL_db = back_propagation(x, h, y, t)

In [ ]:
print("Derivatives wrt w:", dL_dw)
print("Derivatives wrt b1:", dL_db)
print("Derivatives wrt v:", dL_dv)
print("Derivatives wrt c:", dL_dc)

### question4

In [ ]:
from data import load_synth
import random

In [ ]:
(xtrain, ytrain), (xval, yval), num_cls = load_synth()

In [ ]:
def initialize_parameters(input_dim, hidden_dim, output_dim):
    ww = [[random.uniform(-0.1, 0.1) for _ in range(hidden_dim)] for _ in range(input_dim)]
    b = [0.0 for _ in range(hidden_dim)]
    v = [[random.uniform(-0.1, 0.1) for _ in range(output_dim)] for _ in range(hidden_dim)]
    c = [0.0 for _ in range(output_dim)]
    return w, b, v, c

In [ ]:
def min_max_scale(data):
    global_min = min(min(row) for row in data)
    global_max = max(max(row) for row in data)

    # Scale data to [-1, 1] range
    scaled_data = []
    for row in data:
        scaled_row = [2 * ((x - global_min) / (global_max - global_min)) - 1 for x in row]
        scaled_data.append(scaled_row)
    
    return scaled_data

In [ ]:
def convert_to_one_hot(y, num_classes):
    one_hot = [[1 if i == j else 0 for j in range(num_classes)] for i in y]
    return one_hot

In [ ]:
def calculate_loss(y, t):
  loss = 0
  for j in range(len(t)):
    if t[j] == 1:
        loss -= (-1)* math.log(y[j]+ 1e-9) # add a small constant to prevent log(0)
    
  return loss

In [ ]:
def update_parameters(w, b, v, c, dL_dw, dL_db, dL_dv, dL_dc, learning_rate):
    for i in range(len(w)):
        for j in range(len(w[i])):
            w[i][j] -= learning_rate * dL_dw[i][j]
            b[j] -= learning_rate * dL_db[j]
    for i in range(len(v)):
        for j in range(len(v[i])):
            v[i][j] -= learning_rate * dL_dv[i][j]
            c[j] -= learning_rate * dL_dc[j]
    return w, b, v, c

In [ ]:
import matplotlib.pyplot as plt
def train_model(x_train, y_train, num_classes, epochs, learning_rate):
    # Normalize input data
    x_train = min_max_scale(x_train)

    # Convert labels to one-hot encoding
    y_train_one_hot = convert_to_one_hot(y_train, num_classes)

    # Initialize parameters
    w, b, v, c = initialize_parameters(2, 3, 2)

    # Training process
    losses = []
    for x, t in zip(x_train, y_train_one_hot):

        y, h = forward_propagation(x, w, b, v, c)

        
        loss = calculate_loss(y, t)
        losses.append(loss) 

        # backward propagation to get gradients
        dL_dv, dL_dc, dL_dw, dL_db = back_propagation(x, h, y, t)

        # update weights and biases
        w, b, v, c = update_parameters(w, b, v, c, dL_dw, dL_db, dL_dv, dL_dc, learning_rate)

    plt.plot(losses, label='Training Loss')
    plt.xlabel('Training Instances')
    plt.ylabel('Loss')
    plt.title('Loss per Training Instance')
    plt.legend()
    plt.show()


epochs = 100
learning_rate = 0.01

train_model(xtrain, ytrain, num_cls, epochs, learning_rate)